In [32]:
import pandas as pd
import numpy as np
import math
import re
import seaborn as sns
import matplotlib as plt
from matplotlib import pyplot as plt

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [34]:
train =pd.read_csv('train.csv', index_col=0)

In [35]:
train[train['BsmtExposure'].isnull() & train['BsmtQual'].notnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
949,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2006,WD,Normal,192500


In [36]:
import random
random.seed(0)
#obs949 is randomly assigned a a non-"No" categery for BsmtExposure
train['BsmtExposure'][949]=train[train['BsmtExposure']!='No'].sample()['BsmtExposure']

/Users/jameskim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [37]:
#ob with a BsmtFinSF2 but shows "no basement" in the BsmtFinType2 col NEED TO IMPUTE  - random
train[train['BsmtFinType2'].isnull() & train['BsmtFinSF2']!=0]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
333,20,RL,85.0,10655,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,10,2009,WD,Normal,284000


In [38]:
random.seed(0)
#obs333 is randomly assigned a non-"No" categery for BsmtFinType2
train['BsmtFinType2'][333]=train[train['BsmtFinType2']!='nan'].sample()['BsmtFinType2'].values[0]

/Users/jameskim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
#ob with Na in Electrical 
train[train['Electrical'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1380,80,RL,73.0,9735,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,167500


In [40]:
random.seed(0)
#obs1380 is randomly assigned a random
train['Electrical'][1380] = train['Electrical'].sample().values[0]

/Users/jameskim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
#for BsmtExposure - replace No with NoExposure; replace NaN with NoBsmt
train['BsmtExposure']=train['BsmtExposure'].str.replace('No','NoExposure')

In [42]:
train['BsmtExposure'].replace(np.NaN,'NoBsmt',inplace=True)

In [43]:
#GarageYrBlt - change No to numerical zero 
train['GarageYrBlt'].replace(np.NaN,0,inplace=True)

In [44]:
#cols for which NA will be replaced with "No" because NA is a true category thus need to change 
#to "No" so Pandas does not pick it up as np.NaN
FixNA=['Alley', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2','FireplaceQu', 'GarageType','GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [45]:
#change np.NaN to "No"
for v in FixNA:
    train[v].replace(np.NaN,'No',inplace=True)

In [46]:
#spread MiscFeature, replace NaN with zero, and drop No. 
misc=train[['MiscFeature','MiscVal']]
misc=pd.pivot_table(misc,index='Id',columns='MiscFeature',values='MiscVal')
misc.replace(np.NaN,0,inplace=True)
misc=misc.drop(['No','Othr'],axis=1).apply(pd.to_numeric, errors='ignore', downcast='integer')

In [47]:
#combined misc with original DF and drop MiscFeature, MiscVal
train = pd.concat([train.drop(['MiscFeature','MiscVal'],axis=1),misc],axis=1)

In [48]:
train['Electrical'].value_counts()

SBrkr    1335
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [49]:
#list of categorical variables 
cat_variables=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour','Utilities',
               'LotConfig', 'LandSlope','Neighborhood', 'Condition1', 'Condition2',
               'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 
               'Exterior2nd', 'MasVnrType', 'Foundation','BsmtExposure','BsmtFinType2',
               'Heating','CentralAir','Electrical', 'GarageType','SaleType', 'SaleCondition']


In [50]:
#dummify categorical nominal and binary variables
for v in cat_variables:
    temp = pd.get_dummies(train[v],prefix=v, prefix_sep='_')
    temp = temp.drop(temp.iloc[:,0:1],axis=1)
    train = pd.concat([train.drop(v,axis=1),temp],axis=1)


In [51]:
#list for Condition1 and Condition2
Condition1 = list(train.filter(regex='Condition1').columns)
Condition1.remove('Condition1_RRNe')
Condition2 = list(train.filter(regex='Condition2').columns)

In [52]:
#combine all Condition1 and Condition2 columns and drop Condition2
for i,j in zip(Condition1,Condition2):
    train[i] = train[i] | train[j]
    train.drop(j,axis=1,inplace=True)
train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,Gd,TA,...,0,0,0,0,1,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,TA,TA,...,0,0,0,0,1,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,Gd,TA,...,0,0,0,0,1,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,TA,TA,...,0,0,0,0,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,Gd,TA,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,TA,TA,...,0,0,0,0,1,0,0,0,1,0
1457,20,85.0,13175,6,6,1978,1988,119.0,TA,TA,...,0,0,0,0,1,0,0,0,1,0
1458,70,66.0,9042,7,9,1941,2006,0.0,Ex,Gd,...,0,0,0,0,1,0,0,0,1,0


In [53]:
LST = ('ExterQual','ExterCond','HeatingQC','KitchenQual')
LST1 = ('BsmtQual','BsmtCond','FireplaceQu','GarageQual','GarageCond')
LST2 = ('BsmtFinType1')
LST3 = ('Functional')
LST4 = ('GarageFinish')
LST5 = ('PavedDrive')
LST6 = ('PoolQC')
LST7 = ('Fence')


In [54]:
train['TotalBath']=train['BsmtFullBath']+train['BsmtHalfBath']+train['FullBath']+train['HalfBath']
train['Bathroom/Bedroom']=train['TotalBath']/train['BedroomAbvGr']
train[['Bathroom/Bedroom']]
train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBath,Bathroom/Bedroom
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,Gd,TA,...,0,0,1,0,0,0,1,0,4,1.333333
2,20,80.0,9600,6,8,1976,1976,0.0,TA,TA,...,0,0,1,0,0,0,1,0,3,1.000000
3,60,68.0,11250,7,5,2001,2002,162.0,Gd,TA,...,0,0,1,0,0,0,1,0,4,1.333333
4,70,60.0,9550,7,5,1915,1970,0.0,TA,TA,...,0,0,1,0,0,0,0,0,2,0.666667
5,60,84.0,14260,8,5,2000,2000,350.0,Gd,TA,...,0,0,1,0,0,0,1,0,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,TA,TA,...,0,0,1,0,0,0,1,0,3,1.000000
1457,20,85.0,13175,6,6,1978,1988,119.0,TA,TA,...,0,0,1,0,0,0,1,0,3,1.000000
1458,70,66.0,9042,7,9,1941,2006,0.0,Ex,Gd,...,0,0,1,0,0,0,1,0,2,0.500000


In [55]:
for x in LST:
    train.loc[train[x] == 'Po',x] = 1
    train.loc[train[x] == 'Fa',x] = 2
    train.loc[train[x] == 'TA',x] = 3
    train.loc[train[x] == 'Gd',x] = 4
    train.loc[train[x] == 'Ex',x] = 5

In [56]:
for x in LST1:
    train[x].fillna(0, inplace=True)    
    train.loc[train[x] == 'Po',x] = 1
    train.loc[train[x] == 'Fa',x] = 2
    train.loc[train[x] == 'TA',x] = 3
    train.loc[train[x] == 'Gd',x] = 4
    train.loc[train[x] == 'Ex',x] = 5
    

In [57]:
# Nan values mean there are none for this there is no basement therefore value is 0
# all columns for ordinal has no basement or features if filled with 0
x = LST2
train[x].fillna(0, inplace=True)
train.loc[train[x] == 'Unf',x] = 1
train.loc[train[x] == 'LwQ',x] = 2
train.loc[train[x] == 'Rec',x] = 3
train.loc[train[x] == 'BLQ',x] = 4
train.loc[train[x] == 'ALQ',x] = 5
train.loc[train[x] == 'GLQ',x] = 6

In [58]:
x = LST3
train.loc[train[x] == 'Sal',x] = 1
train.loc[train[x] == 'Sev',x] = 2
train.loc[train[x] == 'Maj2',x] = 3
train.loc[train[x] == 'Maj1',x] = 4
train.loc[train[x] == 'Mod',x] = 5
train.loc[train[x] == 'Min2',x] = 6
train.loc[train[x] == 'Min1',x] = 6
train.loc[train[x] == 'Typ',x] = 6

In [59]:
x = LST4
train[x].fillna(0, inplace=True)
train.loc[train[x] == 'Unf',x] = 1
train.loc[train[x] == 'RFn',x] = 2
train.loc[train[x] == 'Fin',x] = 3

    

In [60]:
x = LST5
train.loc[train[x] == 'N',x] = 1
train.loc[train[x] == 'P',x] = 2
train.loc[train[x] == 'Y',x] = 3


In [61]:
x = LST6
train[x].fillna(0, inplace=True)
train.loc[train[x] == 'Fa',x] = 1
train.loc[train[x] == 'TA',x] = 2
train.loc[train[x] == 'Gd',x] = 3
train.loc[train[x] == 'Ex',x] = 4
    

In [62]:
x = LST7
train[x].fillna(0, inplace=True)
train.loc[train[x] == 'MnWw',x] = 1
train.loc[train[x] == 'GdWo',x] = 2
train.loc[train[x] == 'MnPrv',x] = 3
train.loc[train[x] == 'GdPrv',x] = 4    

In [63]:
train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBath,Bathroom/Bedroom
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,4,3,...,0,0,1,0,0,0,1,0,4,1.333333
2,20,80.0,9600,6,8,1976,1976,0.0,3,3,...,0,0,1,0,0,0,1,0,3,1.000000
3,60,68.0,11250,7,5,2001,2002,162.0,4,3,...,0,0,1,0,0,0,1,0,4,1.333333
4,70,60.0,9550,7,5,1915,1970,0.0,3,3,...,0,0,1,0,0,0,0,0,2,0.666667
5,60,84.0,14260,8,5,2000,2000,350.0,4,3,...,0,0,1,0,0,0,1,0,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,3,3,...,0,0,1,0,0,0,1,0,3,1.000000
1457,20,85.0,13175,6,6,1978,1988,119.0,3,3,...,0,0,1,0,0,0,1,0,3,1.000000
1458,70,66.0,9042,7,9,1941,2006,0.0,5,4,...,0,0,1,0,0,0,1,0,2,0.500000


In [67]:
# These are the ordinal columns of the training data set
# train[['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtFinType1','HeatingQC','BsmtFullBath','KitchenQual',
#       'Functional',
# 'FireplaceQu','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','SalePrice']]
#dropping bathroom columns since we made a new column Total Bathroom
train = train.drop('BsmtFullBath',1)
train = train.drop('BsmtHalfBath',1)
train = train.drop('FullBath',1)
train = train.drop('HalfBath',1)

In [68]:
train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBath,Bathroom/Bedroom
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,4,3,...,0,0,1,0,0,0,1,0,4,1.333333
2,20,80.0,9600,6,8,1976,1976,0.0,3,3,...,0,0,1,0,0,0,1,0,3,1.000000
3,60,68.0,11250,7,5,2001,2002,162.0,4,3,...,0,0,1,0,0,0,1,0,4,1.333333
4,70,60.0,9550,7,5,1915,1970,0.0,3,3,...,0,0,1,0,0,0,0,0,2,0.666667
5,60,84.0,14260,8,5,2000,2000,350.0,4,3,...,0,0,1,0,0,0,1,0,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,3,3,...,0,0,1,0,0,0,1,0,3,1.000000
1457,20,85.0,13175,6,6,1978,1988,119.0,3,3,...,0,0,1,0,0,0,1,0,3,1.000000
1458,70,66.0,9042,7,9,1941,2006,0.0,5,4,...,0,0,1,0,0,0,1,0,2,0.500000
